In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
class stylegan(object):
    def __init__(self, session, batch_size=64, output_resolution=256):
        self.sess = session
        self.output_resolution = output_resolution
        self.num_style_blocks = 0
        self.num_discriminator_blocks = 0
        self.num_to_rgbs = 0
        self.num_from_rgbs = 0
        self.num_downsamples = 0
        self.batch_size = batch_size
        
        self.conv_weights = []
        self.w_transforms = []
        
        self.latent_z = tf.random.normal(
            shape=[self.batch_size,512],
            stddev=1.0
        )
        
        # This should come from a feed-forward NN.
        #self.latent_w = tf.placeholder(
        #    shape=[64, 512],
        #    dtype=tf.float32
        #)
        
        self.latent_w = self.latentZMapper(self.latent_z)
        
        self.true_images_ph = tf.placeholder(
            shape=[None, 256, 256, 3],
            dtype=tf.float32
        )
        
        self.generator_out = self.evalGenerator(self.latent_w)
        self.disc_of_gen_out = self.evalDiscriminator(self.generator_out)
        self.disc_of_truth_out = self.evalDiscriminator(self.true_images_ph)
        
        self.discriminator_gradient = None
        
        self.loss = None
        
    def latentZMapper(self, Z_in, depth=8):
        result = None
        for i in range(depth):
            W = tf.get_variable(
                "W_mapper_" + str(i),
                [512, 512],
                initializer=tf.initializers.random_normal(stddev=0.3)
            )
            b = tf.get_variable(
                "b_mapper_" + str(i),
                [512,],
                initializer=tf.initializers.random_normal(stddev=0.3)
            )
            
            if i == 0:
                result = tf.nn.relu(tf.matmul(Z_in, W) + b)
            else:
                result = tf.nn.relu(tf.matmul(result, W) + b)
                
        return result
        
    def evalGenerator(self, W_in):
        with tf.variable_scope("generator", reuse=tf.AUTO_REUSE) as vs:
            self.constant_input = tf.get_variable(
                "c_1",
                [4, 4, 512],
                initializer=tf.initializers.orthogonal
            )
            
            batch_size = tf.shape(W_in)[0]
            
            tiled_constant_input = tf.tile(
                tf.expand_dims(
                    self.constant_input, axis=0
                ),
                [batch_size, 1, 1, 1]
            )
            
            print("tiled constant input:", tiled_constant_input)
            
            block_4_2 = self.styleBlock(
                tiled_constant_input,
                W_in,
                num_input_channels=512,
                #1
                num_output_channels=512,
                fm_dimension=4
            )
            
            to_rgb_1 = self.toRgb(block_4_2, 4, 4, 512)
            
            block_8_1 = self.styleBlock(
                block_4_2,
                W_in,
                num_input_channels=512,
                #2
                num_output_channels=512,
                fm_dimension=8,
                upsample=True
            )
            
            block_8_2 = self.styleBlock(
                block_8_1,
                W_in,
                num_input_channels=512,
                #3
                num_output_channels=512,
                fm_dimension=8
            )
            
            to_rgb_2 = self.toRgb(block_8_2, 8, 8, 512) + self.upsample(to_rgb_1)
            
            block_16_1 = self.styleBlock(
                block_8_2,
                W_in,
                num_input_channels=512,
                #4
                num_output_channels=512,
                fm_dimension=16,
                upsample=True
            )
            
            block_16_2 = self.styleBlock(
                block_16_1,
                W_in,
                num_input_channels=512,
                #5
                num_output_channels=512,
                fm_dimension=16,
            )
            
            to_rgb_3 = self.toRgb(block_16_2, 16, 16, 512) + self.upsample(to_rgb_2)
            
            block_32_1 = self.styleBlock(
                block_16_2,
                W_in,
                num_input_channels=512,
                #6
                num_output_channels=512,
                fm_dimension=32,
                upsample=True
            )
            
            block_32_2 = self.styleBlock(
                block_32_1,
                W_in,
                num_input_channels=512,
                #7
                num_output_channels=512,
                fm_dimension=32,
            )
            
            to_rgb_4 = self.toRgb(block_32_2, 32, 32, 512) + self.upsample(to_rgb_3)
            
            block_64_1 = self.styleBlock(
                block_32_2,
                W_in,
                num_input_channels=512,
                #8
                num_output_channels=512,
                fm_dimension=64,
                upsample=True
            )
            
            block_64_2 = self.styleBlock(
                block_64_1,
                W_in,
                num_input_channels=512,
                num_output_channels=256,
                fm_dimension=64,
            )
            print("block_64_2:", block_64_2)
            to_rgb_5 = self.toRgb(block_64_2, 64, 64, 256) + self.upsample(to_rgb_4)
            
            block_128_1 = self.styleBlock(
                block_64_2,
                W_in,
                num_input_channels=256,
                num_output_channels=256,
                fm_dimension=128,
                upsample=True
            )
            
            block_128_2 = self.styleBlock(
                block_128_1,
                W_in,
                num_input_channels=256,
                num_output_channels=128,
                fm_dimension=128,
            )
            
            to_rgb_6 = self.toRgb(block_128_2, 128, 128, 128) + self.upsample(to_rgb_5)
            
            block_256_1 = self.styleBlock(
                block_128_2,
                W_in,
                num_input_channels=128,
                num_output_channels=128,
                fm_dimension=256,
                upsample=True
            )
            
            block_256_2 = self.styleBlock(
                block_256_1,
                W_in,
                num_input_channels=128,
                num_output_channels=64,
                fm_dimension=256
            )
            
            to_rgb_7 = self.toRgb(block_256_2, 256, 256, 64) + self.upsample(to_rgb_6)
            
            return to_rgb_7
            
    def evalDiscriminator(self, rgb_in):
        with tf.variable_scope("discriminator", reuse=tf.AUTO_REUSE) as vs:
            from_rgb_1 = self.fromRgb(rgb_in, 256, 256, 16)
            
            print("from rgb:", from_rgb_1)
            
            downsample_1 = self.downsample(from_rgb_1, 16)
            block_128_1 = self.discriminatorBlock(
                from_rgb_1,
                16,
                32
            )
            
            res_1 = downsample_1 + block_128_1
            print("res 1:", res_1)
            downsample_2 = self.downsample(res_1, 32)
            block_64_2 = self.discriminatorBlock(
                res_1,
                32,
                64
            )
            
            res_2 = downsample_2 + block_64_2
            
            print("res 2:", res_2)
            
            downsample_3 = self.downsample(res_2, 64)
            block_32_3 = self.discriminatorBlock(
                res_2,
                64,
                128
            )
            
            res_3 = downsample_3 + block_32_3
            
            downsample_4 = self.downsample(res_3, 128)
            block_16_4 = self.discriminatorBlock(
                res_3,
                128,
                256
            )
            
            res_4 = downsample_4 + block_16_4
            
            downsample_5 = self.downsample(res_4, 256)
            block_8_5 = self.discriminatorBlock(
                res_4,
                256,
                512
            )
            
            res_5 = downsample_5 + block_8_5
            
            downsample_6 = self.downsample(res_5, 512, 512)
            block_4_6 = self.discriminatorBlock(
                res_5,
                512,
                512
            )
            
            res_6 = downsample_6 + block_4_6
            
            #downsample_7 = self.downsample(res_6, 512, 512)
            #block_4_7 = self.discriminatorBlock(
            #    res_6,
            #    512,
            #    512
            #)
            #
            #res_7 = downsample_7 + block_4_7
            
            #print("res 7:", res_7)
            
            conv_w_a = tf.get_variable(
                "conv_w_disc_end_3x3",
                [3, 3, 512, 512],
                initializer=tf.initializers.orthogonal
            )
            
            conv_out_1 = tf.nn.leaky_relu(
                tf.nn.conv2d(res_6, conv_w_a, padding="SAME"),
                alpha=0.2
            )
            
            conv_w_b = tf.get_variable(
                "conv_w_disc_end_4x4",
                [4, 4, 512, 512],
                initializer=tf.initializers.orthogonal
            )
            
            conv_out_2 = tf.nn.leaky_relu(
                tf.nn.conv2d(conv_out_1, conv_w_b, padding="VALID"),
                alpha=0.2
            )
            
            batch_size = tf.shape(conv_out_2)[0]
            
            conv_outputs_flat = tf.reshape(conv_out_2, shape=[batch_size, -1])
            
            W_fc = tf.get_variable(
                "fully_connected_W_disc_end",
                [512, 1],
                initializer=tf.initializers.orthogonal
            )
            
            b_fc = tf.get_variable(
                "fully_connected_b_disc_end",
                [1],
                initializer=tf.initializers.random_normal
            )
            
            disc_out = tf.nn.leaky_relu(
                tf.matmul(conv_outputs_flat, W_fc) + b_fc
            )
            
            return disc_out
            
    def discriminatorBlock(self, V_in, num_input_channels, num_output_channels, downsample=True):
        # V_in        --> [batch_size, height, width, num_input_channels]
        # latent_w    --> [batch_size, 512]
        #    num_input_channels  = number of input feature maps
        #    num_output_channels = number of output feature maps
        self.num_discriminator_blocks += 1
        
        conv_weight_a = tf.get_variable(
            "conv_w_disc_a_" + str(self.num_discriminator_blocks),
            [3, 3, num_input_channels, num_input_channels],
            initializer=tf.initializers.orthogonal
        )
        
        V_out_a = tf.nn.leaky_relu(
            tf.nn.conv2d(V_in, conv_weight_a, padding="SAME"),
            alpha=0.2
        )
        
        conv_weight_b = tf.get_variable(
            "conv_w_disc_b_" + str(self.num_discriminator_blocks),
            [3, 3, num_input_channels, num_output_channels],
            initializer=tf.initializers.orthogonal
        )
        
        V_out_b = tf.nn.leaky_relu(
            tf.nn.conv2d(V_out_a, conv_weight_b, padding="SAME"),
            alpha=0.2
        )
        
        V_out = self.downsample(V_out_b, num_output_channels, num_output_channels)
        
        return V_out
            
    def styleBlock(self, V_in, latent_w, num_input_channels, num_output_channels, fm_dimension, upsample=False):
        # V_in        --> [batch_size, height, width, num_input_channels]
        # latent_w    --> [batch_size, 512]
        #    num_input_channels  = number of input feature maps
        #    num_output_channels = number of output feature maps
        self.num_style_blocks += 1
        
        if upsample:
            V_in = self.upsample(V_in)
        
        A = tf.get_variable(
            "A_style" + str(self.num_style_blocks),
            [512, num_input_channels],
            #[512, num_output_channels],
            initializer=tf.initializers.orthogonal
        )
        
        conv_weight = tf.get_variable(
            "conv_w_style" + str(self.num_style_blocks),
            [3, 3, num_input_channels, num_output_channels],
            initializer=tf.initializers.orthogonal
        )
        
        conv_bias = tf.get_variable(
            "conv_b_style" + str(self.num_style_blocks),
            [1, fm_dimension, fm_dimension, num_output_channels],
            initializer=tf.initializers.random_normal
        )
        
        # Affine transformation of latent space vector.
        scale = tf.matmul(latent_w, A)
        
        # Scale input feature map acros input channels by the affine transformation
        # of the latent space input.
        print("##########")
        print("scale input feature map")
        print("scale", scale)
        print("V_in", V_in)
        V_in_scaled = tf.einsum("bi,bhwi->bhwi", scale, V_in)
        
        V_out = tf.nn.conv2d(V_in_scaled, conv_weight, padding="SAME")
        print("V_out:", V_out)
        # This increases the number of weights by a factor of batch_size,
        # which is weird.
        #print("#########")
        print("calculate sigma_j")
        print("scale", scale)
        print("conv_weight", conv_weight)
        #modul_conv_weight = tf.einsum("bc,hwjc->bhwjc", scale, conv_weight)
        modul_conv_weight = tf.einsum("bj,hwjc->bhwjc", scale, conv_weight)
        sigma_j = tf.sqrt(tf.reduce_sum(tf.square(modul_conv_weight), axis=[1, 2, 3]) + 1e-6)
        
        #print("#############")
        print("calculate output")
        print("V_in_scaled", V_in_scaled)
        print("sigma_j", sigma_j)
        # Need to add biases and broadcast noise.
        V_out_scaled = tf.nn.leaky_relu(
            tf.einsum("bhwj,bj->bhwj", V_out, sigma_j) + conv_bias,
            alpha=0.2
        )

        return V_out_scaled
    
    def upsample(self, V_in):
        # Tested with the channel dimension.
        fm_size = tf.shape(V_in)
        batch_size = fm_size[0]
        h = fm_size[1]
        w = fm_size[2]
        c = fm_size[3]
        V_in_a = tf.concat([V_in, V_in,], axis=2)
        V_in_b = tf.reshape(V_in_a, [batch_size, 2*h, w, c])

        V_in_c = tf.transpose(V_in_b, perm=[0, 2, 1, 3])
        V_in_d = tf.concat([V_in_c, V_in_c], axis=2)
        V_out = tf.transpose(tf.reshape(V_in_d, [batch_size, 2*h, 2*w, c]), perm=[0, 2, 1, 3])
        
        return V_out
    
    def downsample(self, V_in, input_channels, output_channels=None):
        self.num_downsamples += 1
        
        if output_channels is None:
            output_channels = 2*input_channels
        
        channel_increase = tf.get_variable(
            "channel_increaser" + str(self.num_downsamples),
            [1, 1, input_channels, output_channels]
        )
        
        V_larger = tf.nn.relu(
            tf.nn.conv2d(V_in, channel_increase, padding="SAME")
        )
        
        V_out = tf.nn.max_pool2d(V_larger, ksize=2, strides=2, padding="VALID")
        return V_out
    
    def toRgb(self, V_in, h, w, c):
        '''
        Convert an NxNxC output block to an RGB image with dimensions
        NxNx3.
        '''
        
        self.num_to_rgbs += 1

        to_rgb = tf.get_variable(
            "to_rgb" + str(self.num_to_rgbs),
            [h, w, c, 3],
            initializer=tf.initializers.random_normal
        )
        print("###############")
        print("V_in:", V_in)
        print("to_rgb:", to_rgb)
        rgb_out = tf.nn.relu(
            tf.nn.conv2d(V_in, to_rgb, padding="SAME")
        )
        
        return rgb_out
    
    def fromRgb(self, V_in, h, w, c):
        '''
        Convert an NxNx3 output block to an feature map with dimensions
        NxNxC.
        '''
        
        self.num_from_rgbs += 1
        
        from_rgb = tf.get_variable(
            "from_rgb" + str(self.num_from_rgbs),
            [h, w, 3, c],
            initializer=tf.initializers.random_normal
        )
        
        feature_map_out = tf.nn.relu(
            tf.nn.conv2d(V_in, from_rgb, padding="SAME")
        )
        
        return feature_map_out

In [3]:
sess = tf.Session()
s = stylegan(sess)

tiled constant input: Tensor("generator/Tile:0", shape=(64, 4, 4, 512), dtype=float32)
##########
scale input feature map
scale Tensor("generator/MatMul:0", shape=(64, 512), dtype=float32)
V_in Tensor("generator/Tile:0", shape=(64, 4, 4, 512), dtype=float32)
V_out: Tensor("generator/Conv2D:0", shape=(64, 4, 4, 512), dtype=float32)
calculate sigma_j
scale Tensor("generator/MatMul:0", shape=(64, 512), dtype=float32)
conv_weight <tf.Variable 'generator/conv_w_style1:0' shape=(3, 3, 512, 512) dtype=float32_ref>
calculate output
V_in_scaled Tensor("generator/einsum/transpose_2:0", shape=(64, 4, 4, 512), dtype=float32)
sigma_j Tensor("generator/Sqrt:0", shape=(64, 512), dtype=float32)
###############
V_in: Tensor("generator/LeakyRelu:0", shape=(64, 4, 4, 512), dtype=float32)
to_rgb: <tf.Variable 'generator/to_rgb1:0' shape=(4, 4, 512, 3) dtype=float32_ref>
##########
scale input feature map
scale Tensor("generator/MatMul_1:0", shape=(64, 512), dtype=float32)
V_in Tensor("generator/transpose_

res 1: Tensor("discriminator/add:0", shape=(64, 128, 128, 32), dtype=float32)
res 2: Tensor("discriminator/add_1:0", shape=(64, 64, 64, 64), dtype=float32)
from rgb: Tensor("discriminator_1/Relu:0", shape=(?, 256, 256, 16), dtype=float32)
res 1: Tensor("discriminator_1/add:0", shape=(?, 128, 128, 32), dtype=float32)
res 2: Tensor("discriminator_1/add_1:0", shape=(?, 64, 64, 64), dtype=float32)


In [4]:
a = tf.constant(
    np.array(
        [
            [[1,2,3],
             [4,5,6],
             [7,8,9]
            ],
            [[-1,-2,-3],
             [-4,-5,-6],
             [-7,-8,-9]
            ],
            [[1.2,2.2,3.2],
             [4.2,5.2,6.2],
             [7.2,8.2,9.2]
            ],
        ]
    )
)

In [5]:
sess = tf.Session()

In [6]:
b = tf.concat([a, a,], axis=2)
c = tf.reshape(b, [3,6,3])

d = tf.transpose(c, perm=[0, 2, 1])
e = tf.concat([d, d], axis=2)
f = tf.transpose(tf.reshape(e, [3, 6, 6]), perm=[0, 2, 1])

g = tf.stack([a, 2*a, 3.4*a], axis=3)
h = tf.concat([g, g], axis=2)
i = tf.reshape(h, [3, 6, 3, 3])
j = tf.transpose(i, perm=[0, 2, 1, 3])
k = tf.concat([j, j], axis=2)
l = tf.transpose(tf.reshape(k, [3, 6, 6, 3]), perm=[0, 2, 1, 3])

for i in range(3):
    print(sess.run(l)[:, :, :, i])

[[[ 1.   1.   2.   2.   3.   3. ]
  [ 1.   1.   2.   2.   3.   3. ]
  [ 4.   4.   5.   5.   6.   6. ]
  [ 4.   4.   5.   5.   6.   6. ]
  [ 7.   7.   8.   8.   9.   9. ]
  [ 7.   7.   8.   8.   9.   9. ]]

 [[-1.  -1.  -2.  -2.  -3.  -3. ]
  [-1.  -1.  -2.  -2.  -3.  -3. ]
  [-4.  -4.  -5.  -5.  -6.  -6. ]
  [-4.  -4.  -5.  -5.  -6.  -6. ]
  [-7.  -7.  -8.  -8.  -9.  -9. ]
  [-7.  -7.  -8.  -8.  -9.  -9. ]]

 [[ 1.2  1.2  2.2  2.2  3.2  3.2]
  [ 1.2  1.2  2.2  2.2  3.2  3.2]
  [ 4.2  4.2  5.2  5.2  6.2  6.2]
  [ 4.2  4.2  5.2  5.2  6.2  6.2]
  [ 7.2  7.2  8.2  8.2  9.2  9.2]
  [ 7.2  7.2  8.2  8.2  9.2  9.2]]]
[[[  2.    2.    4.    4.    6.    6. ]
  [  2.    2.    4.    4.    6.    6. ]
  [  8.    8.   10.   10.   12.   12. ]
  [  8.    8.   10.   10.   12.   12. ]
  [ 14.   14.   16.   16.   18.   18. ]
  [ 14.   14.   16.   16.   18.   18. ]]

 [[ -2.   -2.   -4.   -4.   -6.   -6. ]
  [ -2.   -2.   -4.   -4.   -6.   -6. ]
  [ -8.   -8.  -10.  -10.  -12.  -12. ]
  [ -8.   -8.  -10. 

In [7]:
sess.run(c)

array([[[ 1. ,  2. ,  3. ],
        [ 1. ,  2. ,  3. ],
        [ 4. ,  5. ,  6. ],
        [ 4. ,  5. ,  6. ],
        [ 7. ,  8. ,  9. ],
        [ 7. ,  8. ,  9. ]],

       [[-1. , -2. , -3. ],
        [-1. , -2. , -3. ],
        [-4. , -5. , -6. ],
        [-4. , -5. , -6. ],
        [-7. , -8. , -9. ],
        [-7. , -8. , -9. ]],

       [[ 1.2,  2.2,  3.2],
        [ 1.2,  2.2,  3.2],
        [ 4.2,  5.2,  6.2],
        [ 4.2,  5.2,  6.2],
        [ 7.2,  8.2,  9.2],
        [ 7.2,  8.2,  9.2]]])

In [8]:
sess.run(f)

array([[[ 1. ,  1. ,  2. ,  2. ,  3. ,  3. ],
        [ 1. ,  1. ,  2. ,  2. ,  3. ,  3. ],
        [ 4. ,  4. ,  5. ,  5. ,  6. ,  6. ],
        [ 4. ,  4. ,  5. ,  5. ,  6. ,  6. ],
        [ 7. ,  7. ,  8. ,  8. ,  9. ,  9. ],
        [ 7. ,  7. ,  8. ,  8. ,  9. ,  9. ]],

       [[-1. , -1. , -2. , -2. , -3. , -3. ],
        [-1. , -1. , -2. , -2. , -3. , -3. ],
        [-4. , -4. , -5. , -5. , -6. , -6. ],
        [-4. , -4. , -5. , -5. , -6. , -6. ],
        [-7. , -7. , -8. , -8. , -9. , -9. ],
        [-7. , -7. , -8. , -8. , -9. , -9. ]],

       [[ 1.2,  1.2,  2.2,  2.2,  3.2,  3.2],
        [ 1.2,  1.2,  2.2,  2.2,  3.2,  3.2],
        [ 4.2,  4.2,  5.2,  5.2,  6.2,  6.2],
        [ 4.2,  4.2,  5.2,  5.2,  6.2,  6.2],
        [ 7.2,  7.2,  8.2,  8.2,  9.2,  9.2],
        [ 7.2,  7.2,  8.2,  8.2,  9.2,  9.2]]])

In [9]:
help(tf.reshape)

Help on function reshape in module tensorflow.python.ops.gen_array_ops:

reshape(tensor, shape, name=None)
    Reshapes a tensor.
    
    Given `tensor`, this operation returns a tensor that has the same values
    as `tensor` with shape `shape`.
    
    If one component of `shape` is the special value -1, the size of that dimension
    is computed so that the total size remains constant.  In particular, a `shape`
    of `[-1]` flattens into 1-D.  At most one component of `shape` can be -1.
    
    If `shape` is 1-D or higher, then the operation returns a tensor with shape
    `shape` filled with the values of `tensor`. In this case, the number of elements
    implied by `shape` must be the same as the number of elements in `tensor`.
    
    For example:
    
    ```
    # tensor 't' is [1, 2, 3, 4, 5, 6, 7, 8, 9]
    # tensor 't' has shape [9]
    reshape(t, [3, 3]) ==> [[1, 2, 3],
                            [4, 5, 6],
                            [7, 8, 9]]
    
    # tensor 't' 

In [10]:
help(tf.tile)

Help on function tile in module tensorflow.python.ops.gen_array_ops:

tile(input, multiples, name=None)
    Constructs a tensor by tiling a given tensor.
    
    This operation creates a new tensor by replicating `input` `multiples` times.
    The output tensor's i'th dimension has `input.dims(i) * multiples[i]` elements,
    and the values of `input` are replicated `multiples[i]` times along the 'i'th
    dimension. For example, tiling `[a b c d]` by `[2]` produces
    `[a b c d a b c d]`.
    
    Args:
      input: A `Tensor`. 1-D or higher.
      multiples: A `Tensor`. Must be one of the following types: `int32`, `int64`.
        1-D. Length must be the same as the number of dimensions in `input`
      name: A name for the operation (optional).
    
    Returns:
      A `Tensor`. Has the same type as `input`.



In [11]:
help(tf.transpose)

Help on function transpose in module tensorflow.python.ops.array_ops:

transpose(a, perm=None, name='transpose', conjugate=False)
    Transposes `a`.
    
    Permutes the dimensions according to `perm`.
    
    The returned tensor's dimension i will correspond to the input dimension
    `perm[i]`. If `perm` is not given, it is set to (n-1...0), where n is
    the rank of the input tensor. Hence by default, this operation performs a
    regular matrix transpose on 2-D input Tensors. If conjugate is True and
    `a.dtype` is either `complex64` or `complex128` then the values of `a`
    are conjugated and transposed.
    
    @compatibility(numpy)
    In `numpy` transposes are memory-efficient constant time operations as they
    simply return a new view of the same data with adjusted `strides`.
    
    TensorFlow does not support strides, so `transpose` returns a new tensor with
    the items permuted.
    @end_compatibility
    
    For example:
    
    ```python
    x = tf.constant

In [12]:
help(tf.stack)

Help on function stack in module tensorflow.python.ops.array_ops:

stack(values, axis=0, name='stack')
    Stacks a list of rank-`R` tensors into one rank-`(R+1)` tensor.
    
    Packs the list of tensors in `values` into a tensor with rank one higher than
    each tensor in `values`, by packing them along the `axis` dimension.
    Given a list of length `N` of tensors of shape `(A, B, C)`;
    
    if `axis == 0` then the `output` tensor will have the shape `(N, A, B, C)`.
    if `axis == 1` then the `output` tensor will have the shape `(A, N, B, C)`.
    Etc.
    
    For example:
    
    ```python
    x = tf.constant([1, 4])
    y = tf.constant([2, 5])
    z = tf.constant([3, 6])
    tf.stack([x, y, z])  # [[1, 4], [2, 5], [3, 6]] (Pack along first dim.)
    tf.stack([x, y, z], axis=1)  # [[1, 2, 3], [4, 5, 6]]
    ```
    
    This is the opposite of unstack.  The numpy equivalent is
    
    ```python
    tf.stack([x, y, z]) = np.stack([x, y, z])
    ```
    
    Args:
      val

In [13]:
help(tf.nn.max_pool2d)

Help on function max_pool2d in module tensorflow.python.ops.nn_ops:

max_pool2d(input, ksize, strides, padding, data_format='NHWC', name=None)
    Performs the max pooling on the input.
    
    Args:
      input: A 4-D `Tensor` of the format specified by `data_format`.
      ksize: An int or list of `ints` that has length `1`, `2` or `4`. The size of
        the window for each dimension of the input tensor.
      strides: An int or list of `ints` that has length `1`, `2` or `4`. The
        stride of the sliding window for each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. The padding algorithm. See
        the "returns" section of `tf.nn.convolution` for details.
      data_format: A string. 'NHWC', 'NCHW' and 'NCHW_VECT_C' are supported.
      name: Optional name for the operation.
    
    Returns:
      A `Tensor` of format specified by `data_format`.
      The max pooled output tensor.



In [14]:
help(tf.nn.conv2d)

Help on function conv2d in module tensorflow.python.ops.nn_ops:

conv2d(input, filter=None, strides=None, padding=None, use_cudnn_on_gpu=True, data_format='NHWC', dilations=[1, 1, 1, 1], name=None, filters=None)
    Computes a 2-D convolution given 4-D `input` and `filter` tensors.
    
    Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
    and a filter / kernel tensor of shape
    `[filter_height, filter_width, in_channels, out_channels]`, this op
    performs the following:
    
    1. Flattens the filter to a 2-D matrix with shape
       `[filter_height * filter_width * in_channels, output_channels]`.
    2. Extracts image patches from the input tensor to form a *virtual*
       tensor of shape `[batch, out_height, out_width,
       filter_height * filter_width * in_channels]`.
    3. For each patch, right-multiplies the filter matrix and the image patch
       vector.
    
    In detail, with the default NHWC format,
    
        output[b, i, j, k] =
   

In [15]:
help(tf.concat)

Help on function concat in module tensorflow.python.ops.array_ops:

concat(values, axis, name='concat')
    Concatenates tensors along one dimension.
    
    Concatenates the list of tensors `values` along dimension `axis`.  If
    `values[i].shape = [D0, D1, ... Daxis(i), ...Dn]`, the concatenated
    result has shape
    
        [D0, D1, ... Raxis, ...Dn]
    
    where
    
        Raxis = sum(Daxis(i))
    
    That is, the data from the input tensors is joined along the `axis`
    dimension.
    
    The number of dimensions of the input tensors must match, and all dimensions
    except `axis` must be equal.
    
    For example:
    
    ```python
    t1 = [[1, 2, 3], [4, 5, 6]]
    t2 = [[7, 8, 9], [10, 11, 12]]
    tf.concat([t1, t2], 0)  # [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
    tf.concat([t1, t2], 1)  # [[1, 2, 3, 7, 8, 9], [4, 5, 6, 10, 11, 12]]
    
    # tensor t3 with shape [2, 3]
    # tensor t4 with shape [2, 3]
    tf.shape(tf.concat([t3, t4], 0))  # [4, 

In [16]:
a = tf.tile(tf.expand_dims(tf.constant([2,]), axis=1), [tf.constant(64), 1])

In [17]:
sess.run(a)

array([[2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2]], dtype=int32)

In [18]:
help(tf.get_variable)

Help on function get_variable in module tensorflow.python.ops.variable_scope:

get_variable(name, shape=None, dtype=None, initializer=None, regularizer=None, trainable=None, collections=None, caching_device=None, partitioner=None, validate_shape=True, use_resource=None, custom_getter=None, constraint=None, synchronization=<VariableSynchronization.AUTO: 0>, aggregation=<VariableAggregation.NONE: 0>)
    Gets an existing variable with these parameters or create a new one.
    
    This function prefixes the name with the current variable scope
    and performs reuse checks. See the
    [Variable Scope How To](https://tensorflow.org/guide/variables)
    for an extensive description of how reusing works. Here is a basic example:
    
    ```python
    def foo():
      with tf.variable_scope("foo", reuse=tf.AUTO_REUSE):
        v = tf.get_variable("v", [1])
      return v
    
    v1 = foo()  # Creates v.
    v2 = foo()  # Gets the same, existing v.
    assert v1 == v2
    ```
    
    If i